In [1]:
import pickle
from flask import Flask, jsonify, request,render_template,Response
import flask
import pandas as pd 
from werkzeug.wrappers import Request, Response

In [ ]:
reference : https://youtu.be/UbCWoMf80PY

In [2]:
%%time
app = Flask(__name__)


products = pd.read_csv("data/products.csv")

df_train = pickle.load(open("data_model/train_orders_merged_df_new.pkl","rb"))

df_test = pickle.load(open("data_model/test_orders_merge_df_new.pkl","rb"))

Wall time: 11.2 s


In [3]:
def prediction_function(user_id):
    if user_id in df_train["user_id"].unique():
        data = df_train[df_train["user_id"] == user_id]
        data_ = data.copy()
        data = data.drop(['product_id','order_id',"department_id","aisle_id","reordered"],axis=1)
#         print(data.shape)
        model = pickle.load(open("models_more_features//best_lgbm_new.pkl","rb"))
        data_["prediction"] = model.predict(data)
        d = data_[["user_id","product_id"]]
        d = d.merge(products,on="product_id",how="left")
        recomended_products = []
        count = 0
        for i in d["product_name"]:
            recomended_products.append(i)
            count = count+1
            if (count == 10):
                break
        

    elif user_id in df_test["user_id"].unique():
        data = df_test[df_test["user_id"] == user_id]
        data_ = data.copy()
        data = data.drop(['product_id','order_id',"department_id","aisle_id","order_number"],axis=1)

        model = pickle.load(open("models_more_features//best_lgbm_new.pkl","rb"))
        data_["prediction"] = model.predict(data)
        d = data_[["user_id","product_id"]]
        d = d.merge(products,on="product_id",how="left")
        recomended_products = []
        count = 0
        for i in d["product_name"]:
            recomended_products.append(i)
            count = count+1
            if (count == 10):
                break
    else:
        recomended_products = []
        print("user id is not available in data")
    return recomended_products


In [4]:
@app.route('/')
def index():
    return render_template('index.html')

In [5]:
@app.route('/predict', methods=['POST'])
def predict():
    print("request",request.form.values())
    to_predict_list = [int(x) for x in request.form.values()]
    products = prediction_function(to_predict_list[0])
    output = []
    if len(products) != 0:
        for i in products:
            if i != "None":
                output.append(i)
            else:
                output.append("nothing")
    else:
        output.append("None")
#     print(jsonify(output))
    return Response(render_template('output.html',data=(output)))


In [ ]:
# https://stackoverflow.com/questions/31252791/flask-importerror-no-module-named-flask
# this cell should always be in running
if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Jun/2021 14:00:30] "GET / HTTP/1.1" 200 -


request <generator object MultiDict.values at 0x000002552E98AD60>


127.0.0.1 - - [15/Jun/2021 14:01:02] "POST /predict HTTP/1.1" 200 -


uploaded video of deploytment https://youtu.be/gEKw66jaKQk